In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d.axes3d import get_test_data
from fenics import *
from mshr import *
import numpy as np
import os

In [ ]:
os.chdir('/home/erick/Desktop/Doctorado/doctorado_efsg/pruebas')

In [ ]:
def plot_field_and_fft(field, field_type, fft_field, t): 
    field=np.loadtxt(field,float)
    field = np.rot90(field, k=1, axes=(0, 1))
    fft_field=np.loadtxt(fft_field,float)
    field[field == 0.0] = np.nan

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    
    if field_type == 'c':
        plt.imshow(field, cmap = 'seismic' , interpolation = 'bilinear')
        plt.title("$c(t={})$".format("{:.2f}".format(t)))
    
    if field_type == 's':
        plt.imshow(field, cmap = 'gray' , interpolation = 'bilinear')
        plt.title("$s(t={})$".format("{:.2f}".format(t)))
    
    if field_type == 'i':
        plt.imshow(field, cmap = 'viridis' , interpolation = 'bilinear')
        plt.title("$i(t={})$".format("{:.2f}".format(t)))
    
    plt.colorbar() 
    
    plt.subplot(1, 2, 2)
    plt.imshow(fft_field, cmap = 'seismic' , interpolation = 'bilinear')
    if field_type == 'c':
        plt.title("$c_{fft}(t=%s)$" % "{:.2f}".format(t))
        plt.colorbar()    
        #plt.savefig("fields_c_{}.png".format(t))
        plt.show()
    if field_type == 's':
        plt.title("$s_{fft}(t=%s)$" % "{:.2f}".format(t))
        plt.colorbar()
        #plt.savefig("fields_s_{}.png".format(t))
        plt.show()
    if field_type == 'i':
        plt.title("$i_{fft}(t=%s)$" % "{:.2f}".format(t))
        plt.colorbar()
        #plt.savefig("fields_i_{}.png".format(t))
        plt.show()

In [ ]:
def dft_mean(field, end_step, end_blocks):
    step=0.01
    while step <= end_step:
        final_file = 0.0
        for nb in range(1, end_blocks+1):
            file_name = "matrix_" + field +  "_dft_" + "{:.2f}".format(step) + "_nb_" + str(nb) + ".txt"
            file_load = np.loadtxt(file_name, float)
            final_file += file_load
        final_file = final_file/float(end_blocks)
        #print("step=",step)
        
        f_save = "FFT_" + field + "_" + "{:.2f}".format(step) + ".txt"
        np.savetxt(f_save, final_file, delimiter="\t")
        step += 0.01
    plt.imshow(final_file, cmap=plt.cm.seismic)
    plt.colorbar()
    #plt.savefig("mean_fft.png".format(t))
    plt.show()

In [ ]:
# Time parameters
dt = 0.01 # step
T = 4 # end step
nb = 10

In [ ]:
def power_spectrum(field_type, t, block):
    field_file = "matrix_"  + field_type +"_"+ "{:.2f}".format(t) + "_" + "nb_" + str(block) + ".txt"
    field=np.loadtxt(field_file,float)
    field = np.rot90(field, k=1, axes=(0, 1))
    fft_field = np.fft.fft2(field)
    fft_field=np.fft.fftshift(fft_field)
    fft_field=(np.abs(fft_field))
    max_fft = np.where(fft_field == fft_field.max())
    fft_field[max_fft[0][0],max_fft[1][0]] = 0
    dft_file = "matrix" + "_{}_dft_".format(field_type) + "{:.2f}".format(t) + "_" + "nb_" + str(block) + ".txt"
    np.savetxt(dft_file, fft_field, delimiter="\t")
    return field_file, dft_file

In [ ]:
for block in range(1,nb+1):
    t = 0
    print("block=", block)  
    while t <= T:
        t += dt
        field_c, dft_field_c = power_spectrum('c', t, block)
        field_s, dft_field_s = power_spectrum('s', t, block)
        field_i, dft_field_i = power_spectrum('i', t, block)      
        plot_field_and_fft(field_c, 'c', dft_field_c, t)
        plot_field_and_fft(field_s, 's', dft_field_s, t)
        plot_field_and_fft(field_i, 'i', dft_field_s, t)

In [ ]:
dft_mean('c', T, nb)
dft_mean('s', T, nb)
dft_mean('i', T, nb)